# October 31 - November 1 night time jet

In [10]:
import xarray as xr
import altair as alt
import datetime as dt
alt.data_transformers.enable('json')
import pandas as pd 
import numpy as np
import pytz

from sublimpy import utils, variables, tidy, extrautils

# Download SoS data

In [11]:
download_dir='sosnoqc'
start_date = '20221030'
end_date = '20221104'

In [15]:
sos_data  = utils.download_sos_data(start_date, end_date, variable_names=variables.DEFAULT_VARIABLES)

sos_data = utils.modify_xarray_timezone(sos_data, pytz.UTC, pytz.timezone("US/Mountain"))

# Open NOAA Doppler Lidar data

This was accessed through the web portal

In [17]:
f_list = ['/data2/elilouis/sublimationofsnow/gucdlprofwind4newsM1.c1/gucdlprofwind4newsM1.c1.20221101.000040.nc', '/data2/elilouis/sublimationofsnow/gucdlprofwind4newsM1.c1/gucdlprofwind4newsM1.c1.20221031.000040.nc']
dl_windprofile_data = xr.concat(
    [xr.open_dataset(f) for f in sorted(f_list)],
    dim='time'
)

dl_windprofile_data = utils.modify_xarray_timezone(dl_windprofile_data, pytz.UTC, pytz.timezone("US/Mountain"))

## Hourly wind profiles @ KPS

In [18]:
[v for v in sos_data.data_vars if 'tc_' in v and v.endswith('_c') and len(v) < 10]

['tc_10m_c', 'tc_20m_c', 'tc_3m_c', 'tc_5m_c', 'tc_15m_c', 'tc_2m_c']

# Identify variables we will use

# Extract usable datasets from the raw SoS data

## Extract first-5-minute-hourly-average data from the raw SoS dataset

In [19]:
sos_data_1hr = sos_data.where(
    sos_data['time.minute'] < 6, 
    drop=True
).resample(time='60Min').mean() 

sos_data_1hr_df = tidy.get_tidy_dataset(sos_data_1hr, variable_names=variables.DEFAULT_VARIABLES)

## Extract 5 minute-average data from the raw SoS dataset

In [20]:
sos_data_df = tidy.get_tidy_dataset(sos_data, variable_names=variables.DEFAULT_VARIABLES)

# Examine wind behavior

In [21]:
src = sos_data_1hr_df.copy()

In [22]:
src['day'] = src['time'].dt.day
src['hour'] = src['time'].dt.hour

In [23]:
alt.Chart(src).transform_filter(
    alt.datum.measurement == 'wind speed'
).transform_filter(
    alt.datum.tower == 'c'
).mark_line().encode(
    alt.X('value:Q', sort='-y'),
    alt.Y('height:Q')
).properties(
    width=200,
    height=100,
).facet(
    column=alt.Column(
        'day:O', 
        # header=alt.Header(format="%m/%d %H:00", formatType='time')
    ),
    row=alt.Row(
        'hour:O'
    )
).properties(
    title = 'Hourly average wind profiles'
)

alt.FacetChart(...)

In [25]:
src = sos_data_df.copy()
src['date'] = src['time'].dt.date
src['hour'] = src['time'].dt.hour
src

,time,variable,value,height,tower,measurement,date,hour
0,2022-10-29 18:02:30,tc_1m_uw,-1.364171,1.0,uw,virtual temperature,2022-10-29,18
1,2022-10-29 18:07:30,tc_1m_uw,-1.433663,1.0,uw,virtual temperature,2022-10-29,18
2,2022-10-29 18:12:30,tc_1m_uw,-1.987161,1.0,uw,virtual temperature,2022-10-29,18
3,2022-10-29 18:17:30,tc_1m_uw,-2.118657,1.0,uw,virtual temperature,2022-10-29,18
4,2022-10-29 18:22:30,tc_1m_uw,-2.235014,1.0,uw,virtual temperature,2022-10-29,18
...,...,...,...,...,...,...,...,...
585787,2022-11-04 17:37:30,Rsw_in_9m_d,16.464748,9.0,d,shortwave radiation incoming,2022-11-04,17
585788,2022-11-04 17:42:30,Rsw_in_9m_d,13.491426,9.0,d,shortwave radiation incoming,2022-11-04,17
585789,2022-11-04 17:47:30,Rsw_in_9m_d,11.053195,9.0,d,shortwave radiation incoming,2022-11-04,17
585790,2022-11-04 17:52:30,Rsw_in_9m_d,7.938966,9.0,d,shortwave radiation incoming,2022-11-04,17


In [ ]:
# def create_nighttime_dataframe(start_date, end_date, night_start_hour = 18, night_end_hour = 7):
#     return pd.DataFrame({
#         'start_date': 
#         [
#             dt.datetime(start_date.year, start_date.month, start_date.day, 0)
#         ] + [
#             start_date + dt.timedelta(days=i) + dt.timedelta(hours=night_start_hour) 
#                 for i in range(0, (end_date - start_date).days)
#         ],
#         'end_date': [
#             start_date + dt.timedelta(days = i) + dt.timedelta(hours = night_end_hour) 
#                 for i in range(0, (end_date - start_date).days)] + [dt.datetime(end_date.year, end_date.month, end_date.day, 0)
#         ]
#     })


# night_df = create_nighttime_dataframe(sos_data_df.time.min(), sos_data_df.time.max())
# night_plot = alt.Chart(night_df).mark_rect(color='grey', opacity=0.2).encode(
#     alt.X('start_date:T'),
#     alt.X2('end_date')
# )

In [26]:
local_src = sos_data_1hr_df.copy()

# Create additional time columns to make plotting easier
local_src['day'] = local_src['time'].dt.date.apply(lambda date: dt.datetime.combine(date, dt.time()))
local_src['hour'] = local_src['time'].dt.hour
# local_src = local_src[local_src['time'] < dt.datetime(2022, 11, 4)]
# local_src = local_src[local_src['time'].dt.hour.isin([0,3,6,9,12,15,18,21])]
local_src = local_src[local_src['time'].dt.hour.isin([
    0,4,8,12,16,20,24
    # 0,2,4,6,8,10,12,14,16,18,20,22
])]

profile_plot = alt.Chart(local_src).transform_filter(
    alt.datum.measurement == 'wind speed'
).transform_filter(
    alt.datum.tower == 'c'
).mark_point().encode(
    alt.X(
        'value:Q', 
        sort='-y', 
        title=None, 
        axis=alt.Axis(orient='top', offset=-15)
    ),
    alt.Y('height:Q',  axis=alt.Axis(grid=False)),
    color=alt.Color("height:O", scale=alt.Scale(scheme='viridis'))
).properties(
    width=78,
    height=50
).facet(
    facet=alt.Column(
        'time', 
        # header=alt.Header(format="%m/%d %H:00", formatType='time'),
        header=alt.Header(labelExpr="''"),
        title='Bi-hourly average wind speed profiles (m/s)',
    ),
    columns=24,
   spacing=1
).resolve_axis(
    x='shared'
)

WIND_CHART = alt.Chart(
    sos_data_df
).transform_filter(
    alt.datum.measurement == 'wind direction'
).transform_filter(
    alt.datum.tower == 'c'
).transform_window(
    rolling_mean = 'mean(value)',
    frame = [-5, 5]
).mark_line().encode(
    alt.X(
        'time:T', 
        # axis=alt.Axis(format='%m%d, %H%M')
    ),
    # alt.Y('value:Q'),
    alt.Y(
        'rolling_mean:Q', 
        title = 'Wind Direction (50 minute rolling average)', 
        scale=alt.Scale(domain=[0, 360], nice=False),
        axis=alt.Axis(values=[0, 90, 180, 270, 360]),
        impute=alt.ImputeParams(value=None)
    ),
    color=alt.Color(
        "height:O", 
        scale=alt.Scale(scheme='viridis')
    )
).properties(
    width=1400,
    # height=200
)

In [27]:
(
profile_plot & WIND_CHART
).properties(title='Wind direction and vertical profiles over 2 days')
# .configure_axis(grid=False).configure_view(strokeWidth=0)

alt.VConcatChart(...)

# Slice data to time period of interest


In [90]:
# # Original study periods
########################################################################
########################################################################

# 3 hr period of substantial katabatic behavior
###
# sos_data_df_slice = sos_data_df[
#     (sos_data_df['time'] > dt.datetime(2022, 11, 1, 1))
#     &
#     (sos_data_df['time'] < dt.datetime(2022, 11, 1, 4))
# ]

# sos_data_1hr_df_slice = sos_data_1hr_df[
#     (sos_data_1hr_df['time'] > dt.datetime(2022, 11, 1, 1))
#     &
#     (sos_data_1hr_df['time'] < dt.datetime(2022, 11, 1, 4))
# ]

# 3 hr period of substantial anabatic behavior
###
# sos_data_df_slice = sos_data_df[
#     (sos_data_df['time'] > dt.datetime(2022, 11, 1, 10, 30))
#     &
#     (sos_data_df['time'] < dt.datetime(2022, 11, 1, 13, 30))
# ]

# sos_data_1hr_df_slice = sos_data_1hr_df[
#     (sos_data_1hr_df['time'] > dt.datetime(2022, 11, 1, 10, 30))
#     &
#     (sos_data_1hr_df['time'] < dt.datetime(2022, 11, 1, 13, 30))
# ]




# # Dates aligned with good Doppler LIDAR reads
########################################################################
########################################################################

# 3 hr period of substantial katabatic behavior
###
sos_data_df_slice = sos_data_df[
    (sos_data_df['time'] > dt.datetime(2022, 10, 31, 11))
    &
    (sos_data_df['time'] < dt.datetime(2022, 10, 31, 13))
]

sos_data_1hr_df_slice = sos_data_1hr_df[
    (sos_data_1hr_df['time'] > dt.datetime(2022, 11, 1, 1))
    &
    (sos_data_1hr_df['time'] < dt.datetime(2022, 11, 1, 4))
]

# 3 hr period of substantial anabatic behavior
###
# sos_data_df_slice = sos_data_df[
#     (sos_data_df['time'] > dt.datetime(2022, 10, 31, 12))
#     &
#     (sos_data_df['time'] < dt.datetime(2022, 10, 31, 14))
# ]

# sos_data_1hr_df_slice = sos_data_1hr_df[
#     (sos_data_1hr_df['time'] > dt.datetime(2022, 10, 31, 12))
#     &
#     (sos_data_1hr_df['time'] < dt.datetime(2022, 10, 31, 14))
# ]





# ALL OF IT
########################################################################
########################################################################

# 3 hr period of substantial katabatic behavior
###
# sos_data_df_slice = sos_data_df[
#     (sos_data_df['time'] > dt.datetime(2022, 10, 31, 12))
#     &
#     (sos_data_df['time'] < dt.datetime(2022, 11, 1, 19))
# ]

# sos_data_1hr_df_slice = sos_data_1hr_df[
#     (sos_data_1hr_df['time'] > dt.datetime(2022, 10, 31, 12))
#     &
#     (sos_data_1hr_df['time'] < dt.datetime(2022, 11, 1, 19))
# ]

In [91]:
dl_windprofile_df = dl_windprofile_data.to_dataframe().reset_index()
dl_windprofile_df_slice = dl_windprofile_df[
    (dl_windprofile_df['time'] > dt.datetime(2022, 10, 31, 18))
    &
    (dl_windprofile_df['time'] < dt.datetime(2022, 11, 1, 6))
][['time', 'bound', 'height', 'wind_speed', 'wind_speed_error',
       'wind_direction', 'wind_direction_error']]

## Look at DL and tower winds during the time period

In [92]:
sos_data_df_slice

,time,variable,value,height,tower,measurement
492,2022-10-31 11:02:30,tc_1m_uw,0.212140,1.0,uw,virtual temperature
493,2022-10-31 11:07:30,tc_1m_uw,0.669026,1.0,uw,virtual temperature
494,2022-10-31 11:12:30,tc_1m_uw,0.870984,1.0,uw,virtual temperature
495,2022-10-31 11:17:30,tc_1m_uw,0.975157,1.0,uw,virtual temperature
496,2022-10-31 11:22:30,tc_1m_uw,1.325462,1.0,uw,virtual temperature
...,...,...,...,...,...,...
584575,2022-10-31 12:37:30,Rsw_in_9m_d,672.244446,9.0,d,shortwave radiation incoming
584576,2022-10-31 12:42:30,Rsw_in_9m_d,672.859985,9.0,d,shortwave radiation incoming
584577,2022-10-31 12:47:30,Rsw_in_9m_d,673.098145,9.0,d,shortwave radiation incoming
584578,2022-10-31 12:52:30,Rsw_in_9m_d,673.383667,9.0,d,shortwave radiation incoming


In [93]:
alt.Chart(
    sos_data_df_slice
).transform_filter(
    alt.datum.tower == 'c'
).transform_filter(
    alt.datum.measurement == 'wind direction'
).transform_window(
    rolling_mean = 'mean(value)',
    frame = [0, 0]
).mark_line().encode(
    alt.X('time:T'),
    # alt.Y('value:Q'),
    alt.Y(
        'rolling_mean:Q', 
        title = 'Wind Direction (50 minute rolling average', 
        scale=alt.Scale(domain=[0, 360], nice=False),
        axis=alt.Axis(values=[0, 90, 180, 270, 360])
    ),
    color=alt.Color(
        "height:O", 
        scale=alt.Scale(scheme='viridis')
    )
).properties(
    width=1000,
    # height=200
)
# .configure_axis(grid=False).configure_view(strokeWidth=0)

alt.Chart(...)

In [94]:
local_src = sos_data_1hr_df_slice.copy()

# Create additional time columns to make plotting easier
local_src['day'] = local_src['time'].dt.date.apply(lambda date: dt.datetime.combine(date, dt.time()))
local_src['hour'] = local_src['time'].dt.hour
local_src = local_src[local_src['time'] < dt.datetime(2022, 11, 4)]
# local_src = local_src[local_src['time'].dt.hour.isin([0,3,6,9,12,15,18,21])]


profile_plot = alt.Chart(local_src).transform_filter(
    alt.datum.measurement == 'wind speed'
).mark_point().encode(
    alt.X(
        'value:Q', 
        sort='-y', 
        title=None, 
        axis=alt.Axis(orient='top', offset=-15)
    ),
    alt.Y('height:Q',  axis=alt.Axis(grid=False)),
    color=alt.Color("height:O", scale=alt.Scale(scheme='viridis'))
).properties(
    width=83.4,
    height=50
).facet(
    facet=alt.Column(
        'time', 
        header=alt.Header(format="%H:00", formatType='time'),
        # header=alt.Header(labelExpr="''"),
        title='Hourly average wind speed profiles (m/s)',
    ),
    columns=24,
   spacing=1
).resolve_axis(
    x='shared'
)

(
profile_plot & \
alt.Chart(
    sos_data_df_slice
).transform_filter(
    alt.datum.measurement == 'wind direction'
).transform_window(
    rolling_mean = 'mean(value)',
    frame = [-5, 5]
).mark_line().encode(
    alt.X('time:T'),
    # alt.Y('value:Q'),
    alt.Y(
        'rolling_mean:Q', 
        title = 'Wind Direction (50 minute rolling average', 
        scale=alt.Scale(domain=[0, 360], nice=False),
        axis=alt.Axis(values=[0, 90, 180, 270, 360])

    ),
    color=alt.Color(
        "height:O", 
        scale=alt.Scale(scheme='viridis')
    )
).properties(
    width=1000,
    # height=200
)).properties(title='Wind direction and vertical profiles over 2 days')
# .configure_axis(grid=False).configure_view(strokeWidth=0)

alt.VConcatChart(...)

In [95]:
local_src = sos_data_df_slice.set_index('time').groupby([pd.Grouper(freq='15Min'), 'tower', 'height', 'measurement']).mean().reset_index()

# Create additional time columns to make plotting easier
local_src['day'] = local_src['time'].dt.date.apply(lambda date: dt.datetime.combine(date, dt.time()))
local_src['hour'] = local_src['time'].dt.hour
local_src = local_src[local_src['time'] < dt.datetime(2022, 11, 4)]
# local_src = local_src[local_src['time'].dt.hour.isin([0,3,6,9,12,15,18,21])]

profile_plot = alt.Chart(local_src).transform_filter(
    alt.datum.measurement == 'wind speed'
).transform_filter(
    alt.datum.tower == 'c'
).mark_point().encode(
    alt.X(
        'value:Q', 
        sort='-y', 
        title=None, 
        axis=alt.Axis(orient='top', offset=-30)
    ),
    alt.Y('height:Q',  axis=alt.Axis(grid=False)),
    color=alt.Color("height:O", scale=alt.Scale(scheme='viridis'))
).properties(
    width=81,
    height=50
).facet(
    facet=alt.Column(
        'time', 
        header=alt.Header(format="%H:%M", formatType='time', labelPadding=30, titlePadding=30),
        # header=alt.Header(labelExpr="''"),
        title='15 minute average wind speed profiles (m/s)',
    ),
    columns=24,
   spacing=1
).resolve_axis(
    x='shared'
)

(
profile_plot & \
alt.Chart(
    sos_data_df_slice
).transform_filter(
    alt.datum.measurement == 'wind direction'
).transform_filter(
    alt.datum.tower == 'c'
).transform_window(
    rolling_mean = 'mean(value)',
    frame = [-5, 5]
).mark_line().encode(
    alt.X('time:T'),
    # alt.Y('value:Q'),
    alt.Y(
        'rolling_mean:Q', 
        title = 'Wind Direction (50 minute rolling average', 
        scale=alt.Scale(domain=[0, 360], nice=False),
        axis=alt.Axis(values=[0, 90, 180, 270, 360])

    ),
    color=alt.Color(
        "height:O", 
        scale=alt.Scale(scheme='viridis')
    )
).properties(
    width=1000,
    # height=200
)).properties(title='Wind direction and vertical profiles over 2 days')
# .configure_axis(grid=False).configure_view(strokeWidth=0)

/tmp/ipykernel_26924/3213367688.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  local_src = sos_data_df_slice.set_index('time').groupby([pd.Grouper(freq='15Min'), 'tower', 'height', 'measurement']).mean().reset_index()


alt.VConcatChart(...)

# Convert sonic data to upstream coordinates (ignoring rotation in the z-normal axis, i.e. not adjusting for sloping topography)

This requires averaging over a period of near-constant wind direction. Let's study the second night, 6pm Oct 31 to 6am Nov 1.

## Create rotated dataset in tidy format

In [96]:
sos_data_slice_rot = extrautils.streamwise_coordinates_single_rotation_tidy_df(sos_data_df_slice).query("tower == 'c'")

In [97]:
horzline = alt.Chart().mark_rule().encode(
    y='a:Q'
)

og_plot = alt.Chart(
).mark_point().encode(
    alt.X('time:T'),
    alt.Y('value:Q'),
    alt.Color("height:O", scale=alt.Scale(scheme='viridis')),
    # alt.Facet('measurement', columns=1)
).properties(
    width=300,
    height=100
).transform_filter(
    alt.FieldOneOfPredicate('measurement', ['u', 'v'])
)

alt.layer(
    og_plot,
    horzline, 
    data = sos_data_df_slice.dropna().query("tower == 'c'")
).transform_filter(
    alt.FieldOneOfPredicate('measurement', ['u', 'v'])
).transform_calculate(
        a="0"
).facet('measurement', columns=1) | alt.layer(
    og_plot,
    horzline, 
    data = sos_data_slice_rot.dropna()
).transform_filter(
    alt.FieldOneOfPredicate('measurement', ['u', 'v'])
).transform_calculate(
        a="0"
).facet('measurement', columns=1)

alt.HConcatChart(...)

In [98]:
def grid_plot_with_line(data, columns, xlabel=None):
    
    og_plot = alt.Chart().mark_line(point=False).encode(
        alt.X('value:Q', sort='y', title=xlabel),
        alt.Y('height:Q', title='Height (m)'),
        # alt.Color("height:O", scale=alt.Scale(scheme='viridis')),
    ).properties(
        width=100,
        height=100
    )

    vertline = alt.Chart().mark_rule().encode(
        x='a:Q'
    )

    return alt.layer(
            og_plot, vertline,
            data=data
        ).transform_filter(
           alt.FieldOneOfPredicate('measurement', columns)
        ).transform_calculate(
            a="0"
        ).facet(
            column= alt.Column('time', header=alt.Header(format="%m/%d %H:%M", formatType='time')),
            row = alt.Row('measurement', sort=columns)
        ).resolve_scale(
            y='shared', 
        # x='shared'
        )

def grid_plot_with_line_wxlim(data, columns, xlabel=None, xlim=None):
    
    og_plot = alt.Chart().mark_line(point=False).encode(
        alt.X('value:Q', sort='y', title=xlabel, scale=alt.Scale(domain=xlim)),
        alt.Y('height:Q', title='Height (m)'),
        # alt.Color("height:O", scale=alt.Scale(scheme='viridis')),
    ).properties(
        width=100,
        height=100
    )

    vertline = alt.Chart().mark_rule().encode(
        x='a:Q'
    )

    return alt.layer(
            og_plot, vertline,
            data=data
        ).transform_filter(
           alt.FieldOneOfPredicate('measurement', columns)
        ).transform_calculate(
            a="0"
        ).facet(
            column= alt.Column('time', header=alt.Header(format="%m/%d %H:%M", formatType='time')),
            row = alt.Row('measurement', sort=columns)
        ).resolve_scale(
            y='shared', 
        # x='shared'
        )

In [99]:
src = sos_data_slice_rot.set_index('time').groupby([pd.Grouper(freq='60Min'), 'height', 'tower', 'measurement']).mean().reset_index()

# drop any nans because they mess up plotting continuously
src = src.dropna(subset = 'value')

(
    grid_plot_with_line_wxlim(src, ['temperature'], xlabel='Temperature (C˚)', xlim=[-2,4]) & 
    grid_plot_with_line_wxlim(src, ['u'], xlabel='Wind speed (m/s)', xlim=[0,4]) &
    grid_plot_with_line_wxlim(src, ['u_w_'], xlabel=["Vertical momentum", "flux (m^2/s^2)"], xlim=[-0.03,0.01]) & 
    grid_plot_with_line_wxlim(src, ['w_tc_'], xlabel=["Vertical heat", "flux (m^2 ˚C/s)"], xlim=[-0.03, 0.03]) &
    grid_plot_with_line_wxlim(src, ['w_h2o_'], xlabel=["Vertical moisture", "flux (g/m^2/s)"], xlim=[-0.01,0.01]) &
    grid_plot_with_line_wxlim(src, ['RH'], xlabel=["Relative Humidity (%)"], xlim=[0,70])    
).configure_axis(grid=False)

/tmp/ipykernel_26924/1245099894.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  src = sos_data_slice_rot.set_index('time').groupby([pd.Grouper(freq='60Min'), 'height', 'tower', 'measurement']).mean().reset_index()


alt.VConcatChart(...)